# Grid iterator on Rosenbrock function


In this example, we will use the grid iterator on the [Rosenbrock](https://en.wikipedia.org/wiki/Rosenbrock_function) function.
The Rosenbrock function is a well-known test function from optimization. It serves as the "computational model" for this example.
The grid iterator evaluates a model based on a grid of the input space.

There are two ways of using QUEENS:
1. you can call it in the terminal with an input file and an output directory: `queens --input <path-to-input> --output_dir <output-path>`.
2. you can write a small Python script that you use to run QUEENS.

In this tutorial, we will follow the second approach and work with the script-based workflow.
This allows you to run and interact with QUEENS directly in Jupyter notebooks.

In [ ]:
from queens.distributions.uniform import Uniform
from queens.drivers import Function
from queens.example_simulator_functions.rosenbrock60 import rosenbrock60
from queens.global_settings import GlobalSettings
from queens.iterators.grid import Grid
from queens.main import run_iterator
from queens.models.simulation import Simulation
from queens.parameters.parameters import Parameters
from queens.schedulers import Local
from queens.utils.io import load_result


# Global settings
experiment_name = "grid_iterator_rosenbrock"
output_dir = "./"


with GlobalSettings(
    experiment_name=experiment_name, output_dir=output_dir, debug=False
) as gs:
    #### Model setup ####
    # Model parameters
    x1 = Uniform(lower_bound=-2.0, upper_bound=2.0)
    x2 = Uniform(lower_bound=-2.0, upper_bound=2.0)
    parameters = Parameters(x1=x1, x2=x2)

    # The scheduler manages the execution of your model on your local machine
    scheduler = Local(experiment_name)
    # The driver coordinates the evaluation of the forward model
    driver = Function(parameters=parameters, function=rosenbrock60)
    # The model itself
    # including how it is evaluated (the interface to the computing resource)
    model = Simulation(scheduler, driver)


    #### Analysis setup ####
    # Parameters of the analysis
    grid_design = {
            "x1": {"num_grid_points": 5, "axis_type": "lin", "data_type": "FLOAT"},
            "x2": {"num_grid_points": 5, "axis_type": "lin", "data_type": "FLOAT"},
        }

    result_description ={
            "write_results": True,
            "plot_results": False,
        }
    # The method of the analysis is defined by the iterator type:
    method = Grid(
        grid_design=grid_design,
        result_description=result_description,
        global_settings=gs,
        model=model,
        parameters=parameters,
    )

    #### Analysis ####
    run_iterator(method, gs)

    #### Load Results ####
    result_file = gs.output_dir / f"{gs.experiment_name}.pickle"
    results = load_result(result_file)


## QUEENS output

QUEENS output is written to a file called `<experiment_name>.pickle` in the directory defined by the argument `output_dir` of the global settings object.  
There also the `experiment_name` can be set. In the output folder, you also get a log file that contains the console output. It is called `<experiment_name>.log`.

For this example, the experiment name is set to `grid_iterator_rosenbrock`.  
Therefore, the output can be found in the following directory: `<output_dir>/grid_iterator_rosenbrock.pickle`

In the case of the grid iterator, you get a dictionary including the raw input-output data and some statistics.

In [ ]:
results

### Postprocessing

You can now interact with these results as with any other Python object.
For example, you can plot the results:

In [ ]:
import plotly.graph_objects as go
import numpy as np

input_data = results["input_data"]
output_data = results["raw_output_data"]["result"]

x = np.unique(input_data[:,0])
y = np.unique(input_data[:,1])

shape_x = x.shape[0]
shape_y = y.shape[0]

z = output_data.reshape((shape_x, shape_y))
fig = go.Figure(data=[go.Surface(x=x,y=y, z=z)])

fig.show()

## Time for some individualization
1. The resolution of the grid is relatively coarse.
Increase the resolution by increasing the `num_grid_points` and repeat the simulation.

1. You can adjust the bounds of the grid per variable via the keywords: `lower_bound` and `upper_bound`.  
Go ahead and see what happens if you change the bound.

1. Since we are evaluating a Python function we are using the `Function` Driver. 
Moreover, we are executing the study on our local machine, so we are using the
`Local` Scheduler. You can also run the model evaluations in parallel by 
adjusting its `num_jobs` and `num_procs` parameters.